In [1]:
import cv2
import torch
import numpy as np
import pandas as pd
from Tracker import*
import datetime
from playsound import playsound
import threading

In [2]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt') 

In [3]:
my_file = open("coco.txt", "r") 
data = my_file.read()
class_list = data.split("\n") 

In [4]:
def alert():
    threading.Thread(target = playsound, args=('/Users/mayurpatowary/Desktop/Project/Text to Speech/text_to_speech.mp3',), daemon=True).start()

In [5]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

In [ ]:
#area1 = [(596, 457),(79, 462),(71, 438),(580, 429)]
area1 = [(596, 457),(53, 465), (36, 406), (82, 406),(97, 440), (526, 432), (501, 398),(558, 400)]  
area2 = [(506, 424), (104, 430),(91, 402),(482, 396)] 
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)
cap = cv2.VideoCapture("/Users/mayurpatowary/Desktop/Project/3.mp4")
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 2 != 0:
        continue
    #frame = cv2.resize(frame,(1020,500))
    frame = frame[0:700, 400: 1000] 

    cv2.polylines(frame,[np.array(area1,np.int32)], True, (255,0,0), 2)
    cv2.polylines(frame,[np.array(area2,np.int32)], True, (255,0,0), 2)
    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [5]:
#area2= [(596, 457),(79, 462),(71, 438),(580, 429)] 
area2 = [(596, 457),(53, 465), (36, 406), (82, 406),(97, 440), (526, 432), (501, 398),(558, 400)]  
area1 = [(506, 424), (104, 430),(91, 402),(482, 396)] 
cap = cv2.VideoCapture("/Users/mayurpatowary/Desktop/Project/IOCL_Project/SRU_Unit/Video Files/3.mp4")
count = 0; j = 0
tracker = Tracker()

people_entering = {}
people_exiting = {}

entering = set(); nenter = 0
exiting = set(); nexit = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 4 != 0:
        continue
    #frame = cv2.resize(frame, (1020,500))
    frame = frame[0:700, 400: 1000] 
    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")
    list = []
    for index,row in px.iterrows():
        x1 = int(row[0]) #x coor of starting point 
        y1 = int(row[1]) #y coor of starting point
        x2 = int(row[2]) #x coor of ending point 
        y2 = int(row[3]) #y coor of ending point
        d = int(row[5])
        c = class_list[d] 
        if 'person' in c:
            list.append([x1, y1, x2, y2])
            
    bbox_id = tracker.update(list)
    for bbox in bbox_id:    
        x3, y3, x4, y4, id = bbox
        #Entry
        results = cv2.pointPolygonTest(np.array(area2,np.int32), ((x4, y4)), False)
        if results >= 0:
            people_entering[id] = (x4, y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4), (0,0,255), 2) 
        if id in people_entering:
            results1 = cv2.pointPolygonTest(np.array(area1,np.int32), ((x4, y4)), False)
            if results1 >= 0:
                cv2.rectangle(frame,(x3,y3),(x4,y4), (0,255,0), 2) 
                cv2.circle(frame, (x4, y4), 4, (255,0,255), -1)
                cv2.putText(frame, str(id) ,(x3, y3),cv2.FONT_HERSHEY_COMPLEX,(0.7),(255,255,255),1)
                entering.add(id)
                if j % 100 == 0:
                    alert()
                j += 1
        #Exit
        results2 = cv2.pointPolygonTest(np.array(area1,np.int32), ((x4, y4)), False)
        if results2 >= 0:
            people_exiting[id] = (x4, y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4), (0,255,0), 2) 
        if id in people_exiting:
            results3 = cv2.pointPolygonTest(np.array(area2,np.int32), ((x4, y4)), False)
            if results3 >= 0:
                cv2.rectangle(frame,(x3,y3),(x4,y4), (255,0,255), 2) 
                cv2.circle(frame, (x4, y4), 4, (255,0,255), -1)
                cv2.putText(frame, str(id) ,(x3, y3),cv2.FONT_HERSHEY_COMPLEX,(0.7),(255,255,255),1)
                exiting.add(id)
                
    nenter = len(entering); 
    nexit = len(exiting); 
    nppl = nenter - nexit
    if nenter - nexit == 0:
        cv2.circle(frame, (37, 54), 18, (0,255,0), -1)
    else:
        cv2.circle(frame, (37, 54), 18, (0,0,255), -1)

    cv2.polylines(frame,[np.array(area1,np.int32)], True, (255,0,0), 2)
    cv2.polylines(frame,[np.array(area2,np.int32)], True, (255,0,0), 2)

    current_datetime = datetime.datetime.now()
    time = current_datetime.strftime("%H:%M:%S")
    date = current_datetime.strftime("%Y-%m-%d")
    cv2.putText(frame, time ,(200, 678),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)
    cv2.putText(frame, date ,(360, 678),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)   

    cv2.putText(frame, "Entry: " + str(nenter) ,(54, 637),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)
    cv2.putText(frame, "Exit: " + str(nexit) ,(54, 678),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)        
    cv2.imshow("Frame", frame)
    if time == "00:00:00":
        file_path = "/Users/mayurpatowary/Desktop/Project/Tracks/" + date + '.txt'
        lines = ['Total People Entered today: ' + str(current_nenter), 'Total People Exited today: ' + str(current_nexit)]
        with open(file_path, 'w') as f:
            f.writelines('\n'.join(lines))
        current_nenter = nenter
        current_nexit = nexit
    if time == "00:00:02":
        nenter = 0
        nexit = 0
        exiting = set()
        entering = set()


    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


0: 640x576 1 bicycle, 1 bench, 149.0ms
Speed: 3.9ms preprocess, 149.0ms inference, 11.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 142.7ms
Speed: 1.8ms preprocess, 142.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 135.7ms
Speed: 1.8ms preprocess, 135.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 112.7ms
Speed: 1.5ms preprocess, 112.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 113.4ms
Speed: 1.5ms preprocess, 113.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 114.2ms
Speed: 1.6ms preprocess, 114.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 113.0ms
Speed: 1.7ms preprocess, 113.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x576 1 bicycle, 1 bench, 1

-1

In [60]:
print(px) 
print(row) 
print(c)
print((x1,y1))
print((x2,y2))

           0           1           2           3        4    5
0  164.55191  206.103333  197.030975  393.682404  0.72491  0.0
0    164.551910
1    206.103333
2    197.030975
3    393.682404
4      0.724910
5      0.000000
Name: 0, dtype: float64
person
(164, 206)
(197, 393)


In [59]:
print(type(a))
print(results[0].boxes.data)

<class 'torch.Tensor'>
tensor([[164.5519, 206.1033, 197.0310, 393.6824,   0.7249,   0.0000]])


In [ ]:
rows = np.array(a.shape)[0]
col = np.array(a.shape)[1]
for i in range(0 , rows):
    x1a = int(np.array(a)[i][0])
    y1a = int(np.array(a)[i][1])
    x2a = int(np.array(a)[i][2])
    y2a = int(np.array(a)[i][3])
    da = int(np.array(a)[i][5])
print(x1a)
print(y1a)
print(x2a)
print(y2a)
print(da)
print()

for index,row in px.iterrows():
    x1 = int(row[0]) #x coor of starting point 
    y1 = int(row[1]) #y coor of starting point
    x2 = int(row[2]) #x coor of ending point 
    y2 = int(row[3]) #y coor of ending point
    d = int(row[5])
print(x1)
print(y1)
print(x2)
print(y2)
print(d)

In [63]:
print(type(results[0].boxes)) #[0] is used to go inside results. Inside results, there are many attributes like boxes which we can use.
results[0].boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


<class 'ultralytics.yolo.engine.results.Boxes'>


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[164.5519, 206.1033, 197.0310, 393.6824,   0.7249,   0.0000]])
cls: tensor([0.])
conf: tensor([0.7249])
data: tensor([[164.5519, 206.1033, 197.0310, 393.6824,   0.7249,   0.0000]])
id: None
is_track: False
orig_shape: (500, 350)
shape: torch.Size([1, 6])
xywh: tensor([[180.7914, 299.8929,  32.4791, 187.5791]])
xywhn: tensor([[0.5165, 0.5998, 0.0928, 0.3752]])
xyxy: tensor([[164.5519, 206.1033, 197.0310, 393.6824]])
xyxyn: tensor([[0.4701, 0.4122, 0.5629, 0.7874]])

In [65]:
results

[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair',

In [ ]:
results2 = model(frame)

In [ ]:
cap.release()